In [1]:
import sys

sys.path.insert(0, '../')
from src.mlflow_utils import mlflow

In [2]:
mlflow.set_experiment("Conversational")

In [3]:
import transformers
import torch

model_id = "../Storage/meta-llama_Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

In [4]:
signature = mlflow.models.infer_signature(
    "Hi there, chatbot!",
    mlflow.transformers.generate_signature_output(pipeline, "Hi there, chatbot!"),
)

In [7]:
mlflow.transformers.generate_signature_output(pipeline, "Lorem Ipsum dolor sit amet")

In [12]:
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=pipeline,
        artifact_path="llama-3-8b",
        signature=signature,
        registered_model_name="llama-3-8b-instruct",
    )